In [1]:
# The code was removed by Watson Studio for sharing.

# From a US to a UK Bakery

#### Rolake Odebunmi

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import itertools
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

In [4]:
my_file = project.get_file("survival rate.csv")

# Read the CSV data file from the IBM Watson Studio storage into a pandas DataFrame
my_file.seek(0)
df_business=pd.read_csv(my_file)
df_business.columns=['Authority',"Actives 2017","Births 2017","Deaths 2017","Survival Rate"]
df_business.head()

,Authority,Actives 2017,Births 2017,Deaths 2017,Survival Rate
0,United Kingdom,2925600.0,381885.0,356815.0,12.20
1,Durham,13870.0,1610.0,1510.0,10.89
2,Darlington,3490.0,440.0,450.0,12.89
3,Hartlepool,2695.0,320.0,375.0,13.91
4,Middlesbrough,3670.0,515.0,490.0,13.35


In [10]:
df_bus = df_business.loc[df_business['Authority'].isin(['Inner London','Outer London','City of London','City of Edinburgh','Glasgow City','Greater Manchester','Birmingham','Liverpool','Bristol City of','Cambridge','Oxford','Brighton and Hove'])] 
df_bus.reset_index(inplace = True)
df_bus.sort_values(by=['Survival Rate'])
df_bus

,index,Authority,Actives 2017,Births 2017,Deaths 2017,Survival Rate
0,27,Greater Manchester,124385.0,23590.0,16340.0,13.14
1,53,Liverpool,17020.0,3735.0,2315.0,13.60
2,146,Birmingham,43520.0,6770.0,6740.0,15.49
3,167,Cambridge,5550.0,635.0,580.0,10.45
4,212,Inner London,323425.0,50800.0,48155.0,14.89
5,214,City of London,32990.0,5905.0,7395.0,22.42
6,227,Outer London,284445.0,41500.0,38115.0,13.40
7,248,Brighton and Hove,15910.0,2110.0,1840.0,11.57
8,297,Oxford,5385.0,610.0,595.0,11.05
9,388,City of Edinburgh,21400.0,2640.0,2495.0,11.66


Based on the survival rates shown above, the top 5 areas are:

    1. City of London
    2. Birmingham
    3. Inner London
    4. Liverpool
    5. Glasgow City

As this list includes 2 areas of London, we will look at the best area for London which is City of London and then the four other locations in UK.

Now we will analyse these 4 areas using Foursquare to explore the locations and popularity/number of other bakeries in the area.

In [11]:
# The code was removed by Watson Studio for sharing.

Your credentials:
CLIENT_ID: provided and hidden for safety
CLIENT_SECRET: provided and hidden for safety


In [12]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [13]:
addr1 = 'City of London, UK'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(addr1)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.5156177 -0.0919983


In [19]:
search_query = 'Bakery'
radius = 5000
LIMIT=50
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
results1 = requests.get(url).json()
results1

{'meta': {'code': 200, 'requestId': '5d7a97c3e0c0c9002c8a7d7d'},
 'response': {'venues': [{'id': '56025d64498e1f6977dbb1e1',
    'name': "GAIL's Bakery",
    'location': {'address': '70 Holland St',
     'crossStreet': 'Unit C Neo-Bankside',
     'lat': 51.50680605097336,
     'lng': -0.10060863952807352,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.50680605097336,
       'lng': -0.10060863952807352}],
     'distance': 1148,
     'postalCode': 'SE1 9NX',
     'cc': 'GB',
     'city': 'South Bank',
     'state': 'Greater London',
     'country': 'United Kingdom',
     'formattedAddress': ['70 Holland St (Unit C Neo-Bankside)',
      'South Bank',
      'Greater London',
      'SE1 9NX',
      'United Kingdom']},
    'categories': [{'id': '4bf58dd8d48988d16a941735',
      'name': 'Bakery',
      'pluralName': 'Bakeries',
      'shortName': 'Bakery',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
       'suffix': '.png'},
      'primary': 

In [20]:
# assign relevant part of JSON to venues
venues = results1['response']['venues']
# tranform venues into a dataframe
dataframe1 = json_normalize(venues)
dataframe1.head()
# keep only columns that include venue name, and anything that is associated with location
filtered_columns1 = ['name', 'categories'] + [col for col in dataframe1.columns if col.startswith('location.')] + ['id']
dataframe_filtered1 = dataframe1.loc[:, filtered_columns1]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# filter the category for each row
dataframe_filtered1['categories'] = dataframe_filtered1.apply(get_category_type, axis=1)
# clean column names by keeping only last term
dataframe_filtered1.columns = [column.split('.')[-1] for column in dataframe_filtered1.columns]
dataframe_filtered1
dataframe_filtered1.name

0                          GAIL's Bakery
1                          GAIL's Bakery
2                    The Lebanese Bakery
3                 A.O.K Kitchen & Bakery
4                 The Hummingbird Bakery
5                  Granier Bakery & Cafe
6                          GAIL's Bakery
7                       Olivier's Bakery
8                               B Bakery
9                  Original Bagel Bakery
10                         GAIL's Bakery
11                         GAIL's Bakery
12                         GAIL's Bakery
13                         Benita Bakery
14                         GAIL's Bakery
15                      Euphorium Bakery
16                        St John Bakery
17                       St. John Bakery
18                      Euphorium Bakery
19                         GAIL's Bakery
20                         GAIL's Bakery
21                The Hummingbird Bakery
22                Dominique Ansel Bakery
23                         GAIL's Bakery
24              

In [21]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 
# add a red circle marker to represent the location
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='City of London',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)
# add the bakeries as blue circle markers
for lat, lng, label in zip(dataframe_filtered1.lat, dataframe_filtered1.lng, dataframe_filtered1.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map


In [23]:
#Explore Hoxton
latitude = 51.527271
longitude = -0.080090
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url
import requests
results1 = requests.get(url).json()
items1 = results['response']['groups'][0]['items']
items1[0]
df1 = json_normalize(items1) # flatten JSON
# filter columns
filt_col1 = ['venue.name', 'venue.categories'] + [col for col in df1.columns if col.startswith('venue.location.')] + ['venue.id']
df_filt1 = df1.loc[:, filt_col1]

# filter the category for each row
df_filt1['venue.categories'] = df_filt1.apply(get_category_type, axis=1)

# clean columns
df_filt1.columns = [col.split('.')[-1] for col in df_filt1.columns]

df_filt1

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,National Centre for Circus Arts,Performing Arts Venue,Coronet St,GB,London,United Kingdom,NaN,173,"[Coronet St, London, Greater London, N1 6HD, U...","[{'label': 'display', 'lat': 51.52757805817418...",51.527578,-0.082551,NaN,N1 6HD,Greater London,4acfdf51f964a52093d620e3
1,Happiness Forgets,Cocktail Bar,8-9 Hoxton Sq,GB,Hackney,United Kingdom,Basement,116,"[8-9 Hoxton Sq (Basement), Hackney, Greater Lo...","[{'label': 'display', 'lat': 51.527746, 'lng':...",51.527746,-0.081591,NaN,N1 6NU,Greater London,4d000afb21ea6ea817ce3f9f
2,Homeslice Pizza,Pizza Place,374-378 Old St,GB,London,United Kingdom,NaN,33,"[374-378 Old St, London, Greater London, EC1V ...","[{'label': 'display', 'lat': 51.52710033, 'lng...",51.527100,-0.079689,NaN,EC1V 9LT,Greater London,565f5a3e498e305de0a17f07
3,Origin Coffee,Coffee Shop,65 Charlotte Rd,GB,London,United Kingdom,NaN,128,"[65 Charlotte Rd, London, Greater London, EC2A...","[{'label': 'display', 'lat': 51.52634701464375...",51.526347,-0.081204,"Shoreditch, London, Greater London",EC2A 3PE,Greater London,556dba64498e452733b375a0
4,Dishoom,Indian Restaurant,7 Boundary St,GB,London,United Kingdom,NaN,380,"[7 Boundary St, London, Greater London, E2 7JE...","[{'label': 'display', 'lat': 51.52451482185366...",51.524515,-0.076850,Shoreditch,E2 7JE,Greater London,5040721ae4b01446aa41d438
5,Dinerama,Street Food Gathering,19 Great Eastern St,GB,London,United Kingdom,NaN,450,"[19 Great Eastern St, London, Greater London, ...","[{'label': 'display', 'lat': 51.52326108268971...",51.523261,-0.079233,Hackney,EC2A 3EJ,Greater London,5579efae498e50edff5b18d1
6,BunBunBun Vietnamese Food,Vietnamese Restaurant,134B Kingsland Rd,GB,London,United Kingdom,NaN,457,"[134B Kingsland Rd, London, Greater London, E2...","[{'label': 'display', 'lat': 51.53093012782418...",51.530930,-0.077084,NaN,E2 8DY,Greater London,54086528498e4f691ec091b3
7,Burro e Salvia,Italian Restaurant,52 Redchurch St,GB,London,United Kingdom,NaN,494,"[52 Redchurch St, London, Greater London, E2 7...","[{'label': 'display', 'lat': 51.52442972995472...",51.524430,-0.074598,NaN,E2 7DP,Greater London,51a62209454ae7b27328f944
8,The Nightjar,Cocktail Bar,129 City Rd,GB,London,United Kingdom,NaN,545,"[129 City Rd, London, Greater London, EC1V 1JB...","[{'label': 'display', 'lat': 51.52645036141374...",51.526450,-0.087860,NaN,EC1V 1JB,Greater London,4c9a25e613de95212f13d653
9,Ozone Coffee Roasters,Coffee Shop,11 Leonard St,GB,London,United Kingdom,NaN,542,"[11 Leonard St, London, Greater London, EC2A 4...","[{'label': 'display', 'lat': 51.52469267882992...",51.524693,-0.086737,"Shoreditch, London, Greater London",EC2A 4AQ,Greater London,4f54937ce4b0e14ed9512da2


In [24]:
#Explore Bethnal Green
latitude = 51.529331
longitude = -0.055910
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url
import requests
results1a = requests.get(url).json()
items1a = results1a['response']['groups'][0]['items']
items1a[0]
df1a = json_normalize(items1a) # flatten JSON
# filter columns
filt_col1a = ['venue.name', 'venue.categories'] + [col for col in df1a.columns if col.startswith('venue.location.')] + ['venue.id']
df_filt1a = df1a.loc[:, filt_col1a]

# filter the category for each row
df_filt1a['venue.categories'] = df_filt1a.apply(get_category_type, axis=1)

# clean columns
df_filt1a.columns = [col.split('.')[-1] for col in df_filt1a.columns]

df_filt1a

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Mother Kelly's Bottle Shop and Tap Room,Beer Bar,251 Paradise Row,GB,London,United Kingdom,NaN,102,"[251 Paradise Row, London, Greater London, E2 ...","[{'label': 'display', 'lat': 51.52841340778298...",51.528413,-0.055843,NaN,E2 9LE,Greater London,53485a14498e376571ea9795
1,Satan's Whiskers,Cocktail Bar,343 Cambridge Heath Rd,GB,London,United Kingdom,NaN,29,"[343 Cambridge Heath Rd, London, Greater Londo...","[{'label': 'display', 'lat': 51.52958839622787...",51.529588,-0.055997,NaN,E2 9RA,Greater London,526a5bd611d2be4d57a79b8c
2,The King's Arms,Pub,11a Buckfast St,GB,Bethnal Green,United Kingdom,NaN,796,"[11a Buckfast St, Bethnal Green, Greater Londo...","[{'label': 'display', 'lat': 51.52575390729900...",51.525754,-0.065868,NaN,E2 6EY,Greater London,52b47ed2498e43984ea0c7de
3,The Virgin Queen,Pub,Goldsmiths Row,GB,London,United Kingdom,Goldsmiths Square,763,"[Goldsmiths Row (Goldsmiths Square), London, G...","[{'label': 'display', 'lat': 51.53368033383224...",51.533680,-0.064435,NaN,E2 8QY,Greater London,5a29aace4b78c5531da5520b
4,Broadway Market,Market,Broadway Market,GB,London,United Kingdom,Westgate St,905,"[Broadway Market (Westgate St), London, Greate...","[{'label': 'display', 'lat': 51.536596, 'lng':...",51.536596,-0.061796,NaN,E8 4QJ,Greater London,4ac518d2f964a5200aa720e3
5,The Broadway Bookshop,Bookstore,6 Broadway Market,GB,Hackney,United Kingdom,NaN,841,"[6 Broadway Market, Hackney, Greater London, E...","[{'label': 'display', 'lat': 51.5358209375085,...",51.535821,-0.062136,NaN,E8 4QJ,Greater London,4c669a0e9cb82d7f94f390d2
6,"Acton's Lock, Regent's Canal",Canal Lock,Andrew's Road,GB,London,United Kingdom,at Broadway Market,879,"[Andrew's Road (at Broadway Market), London, G...","[{'label': 'display', 'lat': 51.53559353690913...",51.535594,-0.063663,NaN,E2 9AP,Greater London,4bb74c672ea195211ebaac2f
7,Tuyo,Mediterranean Restaurant,129A Pritchard’s Rd,GB,Hackney,United Kingdom,NaN,821,"[129A Pritchard’s Rd, Hackney, Greater London,...","[{'label': 'display', 'lat': 51.53540083343848...",51.535401,-0.062669,Bethnal Green North,E2 9AP,Greater London,5961e87c35d3fc3e621cee43
8,Stretch,Yoga Studio,6b Ada St,GB,London,United Kingdom,Broadway Market,817,"[6b Ada St (Broadway Market), London, Greater ...","[{'label': 'display', 'lat': 51.53580233790787...",51.535802,-0.061478,NaN,E8 4QU,Greater London,51bb225dabd8896db992e37c
9,Artwords Bookshop,Bookstore,20-22 Broadway Market,GB,Hackney,United Kingdom,NaN,875,"[20-22 Broadway Market, Hackney, Greater Londo...","[{'label': 'display', 'lat': 51.53623559069535...",51.536236,-0.061952,London Fields,E8 4QJ,Greater London,4ca87fd8d971b1f7e5eb14e1


In [25]:
#Explore King's Cross
latitude = 51.528450
longitude = -0.122700
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url
import requests
results1b = requests.get(url).json()
items1b = results1b['response']['groups'][0]['items']
items1b[0]
df1b = json_normalize(items1b) # flatten JSON
# filter columns
filt_col1b = ['venue.name', 'venue.categories'] + [col for col in df1b.columns if col.startswith('venue.location.')] + ['venue.id']
df_filt1b = df1b.loc[:, filt_col1b]

# filter the category for each row
df_filt1b['venue.categories'] = df_filt1b.apply(get_category_type, axis=1)

# clean columns
df_filt1b.columns = [col.split('.')[-1] for col in df_filt1b.columns]

df_filt1b

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Aux Pains de Papy,Bakery,279 Grays Inn Rd,GB,London,United Kingdom,NaN,193,"[279 Grays Inn Rd, London, Greater London, WC1...","[{'label': 'display', 'lat': 51.52934, 'lng': ...",51.529340,-0.120303,King's Cross,WC1X 8QF,Greater London,5200a560498ec1b0f59f7eb0
1,The Sir John Ritblat Gallery: Treasures of the...,Museum,96 Euston Road,GB,London,United Kingdom,The British Library,361,"[96 Euston Road (The British Library), London,...","[{'label': 'display', 'lat': 51.52966611994934...",51.529666,-0.127541,NaN,NW1 2DB,Greater London,4daeaa490437710b8137b098
2,M&S Simply Food,Grocery Store,St Pancras International Station,GB,London,United Kingdom,The Circle,344,[St Pancras International Station (The Circle)...,"[{'label': 'display', 'lat': 51.53096383372583...",51.530964,-0.125591,NaN,NW1 2QP,Greater London,4af53b8df964a5209ef821e3
3,Pizza Union,Pizza Place,246-250 Pentonville Rd,GB,London,United Kingdom,Kings Cross Rd,341,"[246-250 Pentonville Rd (Kings Cross Rd), Lond...","[{'label': 'display', 'lat': 51.53098437323851...",51.530984,-0.119933,Caledonian,N1 9JY,Greater London,5570465a498eb53da6d0e9d4
4,Barry's Bootcamp,Gym / Fitness Center,163 Euston Rd,GB,London,United Kingdom,at Upper Woburn Pl,561,"[163 Euston Rd (at Upper Woburn Pl), Camden, G...","[{'label': 'display', 'lat': 51.52712140360649...",51.527121,-0.130528,King's Cross,NW1 2BH,Greater London,5103ff0ae4b081fc416f95bd
5,Granary Square,Plaza,Granary Sq,GB,London,United Kingdom,NaN,765,"[Granary Sq, London, Greater London, N1C, Uni...","[{'label': 'display', 'lat': 51.53517858597692...",51.535179,-0.125010,King's Cr,N1C,Greater London,4e80a8a2d3e33fe0d8c59334
6,Dishoom,Indian Restaurant,5 Stable St,GB,London,United Kingdom,NaN,874,"[5 Stable St, London, Greater London, N1C 4AB,...","[{'label': 'display', 'lat': 51.53608035111114...",51.536080,-0.125733,King's Cross,N1C 4AB,Greater London,53cd09ab498e5a7d7b19b4c1
7,British Museum,History Museum,Great Russell St,GB,London,United Kingdom,btwn Montague & Bloomsbury St,1082,[Great Russell St (btwn Montague & Bloomsbury ...,"[{'label': 'display', 'lat': 51.51900874337275...",51.519009,-0.126437,NaN,WC1B 3DG,Greater London,4ac518d2f964a5203da720e3
8,Sadler's Wells,Theater,Rosebery Ave,GB,London,United Kingdom,NaN,1138,"[Rosebery Ave, London, Greater London, EC1R 4T...","[{'label': 'display', 'lat': 51.52907838308066...",51.529078,-0.106294,NaN,EC1R 4TN,Greater London,4ac518e8f964a52077ab20e3
9,Scarfes Bar,Hotel Bar,252 High Holborn,GB,London,United Kingdom,NaN,1224,"[252 High Holborn, London, Greater London, WC1...","[{'label': 'display', 'lat': 51.51781314398572...",51.517813,-0.118184,NaN,WC1V 7EN,Greater London,5261511311d2d7cfe4189803


In [27]:
#Ratings
venue_id = '4d875f6802eb5481092269f5' # ID of E5 Bakehouse
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url
result_e5 = requests.get(url).json()
print(result_e5['response']['venue'].keys())
result_e5['response']['venue']
try:
    print(result_e5['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')


dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])
9.2


In [28]:
#Ratings
venue_id = '5200a560498ec1b0f59f7eb0' # ID of Aux pains de Papy
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url
result_aux = requests.get(url).json()
print(result_aux['response']['venue'].keys())
result_aux['response']['venue']
try:
    print(result_aux['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])
9.1


In [29]:
#Ratings
venue_id = '560d0db9498e57efd528f261' # ID of Fabrique
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url
result_fab = requests.get(url).json()
print(result_fab['response']['venue'].keys())
result_fab['response']['venue']
try:
    print(result_fab['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'price', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])
9.2


The above steps done for City of London will then be repeated for the other top 3 areas (excluding Inner London).

##### Birmingham

In [30]:
addr2 = 'Birmingham, UK'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(addr2)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

52.4775396 -1.894053


In [31]:
search_query = 'Bakery'
radius = 5000
LIMIT=50
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
results2 = requests.get(url).json()

In [32]:
# assign relevant part of JSON to venues
venues = results2['response']['venues']
# tranform venues into a dataframe
dataframe2 = json_normalize(venues)
dataframe2.head()
# keep only columns that include venue name, and anything that is associated with location
filtered_columns2 = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')] + ['id']
dataframe_filtered2 = dataframe2.loc[:, filtered_columns2]
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
# filter the category for each row
dataframe_filtered2['categories'] = dataframe_filtered2.apply(get_category_type, axis=1)
# clean column names by keeping only last term
dataframe_filtered2.columns = [column.split('.')[-1] for column in dataframe_filtered2.columns]

In [33]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 
# add a red circle marker to represent the location
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Birmingham',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)
# add the bakeries as blue circle markers
for lat, lng, label in zip(dataframe_filtered2.lat, dataframe_filtered2.lng, dataframe_filtered2.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [34]:
#Explore Ladywood
latitude = 52.480595
longitude = -1.918506
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
results2a = requests.get(url).json()
items2a = results2a['response']['groups'][0]['items']
items2a[0]
df2a = json_normalize(items2a) # flatten JSON
# filter columns
filt_col2a = ['venue.name', 'venue.categories'] + [col for col in df2a.columns if col.startswith('venue.location.')] + ['venue.id']
df_filt2a = df2a.loc[:, filt_col2a]
# filter the category for each row
df_filt2a['venue.categories'] = df_filt2a.apply(get_category_type, axis=1)
# clean columns
df_filt2a.columns = [col.split('.')[-1] for col in df_filt2a.columns]
df_filt2a

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Brindleyplace,Shopping Plaza,2 Brunswick St,GB,Birmingham,United Kingdom,NaN,471,"[2 Brunswick St, Birmingham, West Midlands, B1...","[{'label': 'display', 'lat': 52.47790439170848...",52.477904,-1.913134,NaN,B1 2JF,West Midlands,4b0933d0f964a520b41423e3
1,The Pig & Tail,Pub,12-13 Albion St,GB,Birmingham,United Kingdom,NaN,529,"[12-13 Albion St, Birmingham, West Midlands, B...","[{'label': 'display', 'lat': 52.48462681533808...",52.484627,-1.914357,Jewellery Quarter,B1 3AH,West Midlands,575c29a9498e4764be047042
2,1000 Trades,Bar,16 Frederick St,GB,Birmingham,United Kingdom,NaN,694,"[16 Frederick St, Birmingham, West Midlands, B...","[{'label': 'display', 'lat': 52.485203, 'lng':...",52.485203,-1.911601,NaN,B1 3HE,West Midlands,575d7cac498e53845864606f
3,Rajdoot Tandoori,Indian Restaurant,78-79 George St.,GB,Birmingham,United Kingdom,NaN,585,"[78-79 George St., Birmingham, West Midlands, ...","[{'label': 'display', 'lat': 52.48263488483381...",52.482635,-1.910547,NaN,B3 1PY,West Midlands,4bb4f6cf29269c743be3ca92
4,Symphony Hall,Concert Hall,Centenary Sq,GB,Birmingham,United Kingdom,at Broad St,592,"[Centenary Sq (at Broad St), Birmingham, West ...","[{'label': 'display', 'lat': 52.47887388988232...",52.478874,-1.910229,NaN,B1 2EA,West Midlands,4bc8b98a14d7952117b569e9
5,The Flapper,Pub,Cambrian Wharf,GB,Birmingham,United Kingdom,Off Kingston Row,437,"[Cambrian Wharf (Off Kingston Row), Birmingham...","[{'label': 'display', 'lat': 52.48051878039794...",52.480519,-1.912056,NaN,B1 2NU,NaN,4b05882af964a520b9b522e3
6,Canalside Walk,Trail,NaN,GB,Birmingham,United Kingdom,NaN,640,"[Birmingham, West Midlands, United Kingdom]","[{'label': 'display', 'lat': 52.48131052691651...",52.481311,-1.909139,NaN,NaN,West Midlands,51dfe1f7498ef2f8777b92b1
7,Hyatt Regency Birmingham,Hotel,2 Bridge St,GB,Birmingham,United Kingdom,NaN,681,"[2 Bridge St, Birmingham, West Midlands, B1 2J...","[{'label': 'display', 'lat': 52.47809589017726...",52.478096,-1.909324,NaN,B1 2JZ,West Midlands,4b058829f964a5206cb522e3
8,Otto Pizza,Pizza Place,14 Caroline St,GB,Birmingham,United Kingdom,NaN,975,"[14 Caroline St, Birmingham, West Midlands, B3...","[{'label': 'display', 'lat': 52.48640636459643...",52.486406,-1.907743,Jewellery Quarter,B3 1TR,West Midlands,57765039498ee216f850c3c2
9,The Button Factory,Bar,25 Frederick St,GB,Birmingham,United Kingdom,NaN,739,"[25 Frederick St, Birmingham, West Midlands, B...","[{'label': 'display', 'lat': 52.48587101647327...",52.485871,-1.911885,NaN,B1 3HH,West Midlands,576ac89d498ea5b6bff40c87


In [36]:
#Explore Aston University
latitude = 52.486210
longitude = -1.888499
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
results2b = requests.get(url).json()
items2b = results2b['response']['groups'][0]['items']
items2b[0]
df2b = json_normalize(items2b) # flatten JSON
# filter columns
filt_col2b = ['venue.name', 'venue.categories'] + [col for col in df2b.columns if col.startswith('venue.location.')] + ['venue.id']
df_filt2b = df2b.loc[:, filt_col2b]
# filter the category for each row
df_filt2b['venue.categories'] = df_filt2b.apply(get_category_type, axis=1)
# clean columns
df_filt2b.columns = [col.split('.')[-1] for col in df_filt2b.columns]
df_filt2b

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The Sir Doug Ellis Woodcock Sports Centre,Gym / Fitness Center,Woodcock Street,GB,Birmingham,United Kingdom,NaN,146,"[Woodcock Street, Birmingham, West Midlands, B...","[{'label': 'display', 'lat': 52.48619189550715...",52.486192,-1.886339,NaN,B4 7ET,West Midlands,4b6f08c9f964a52056d82ce3
1,Boston Tea Party,Café,190 Corporation St,GB,Birmingham,United Kingdom,NaN,489,"[190 Corporation St, Birmingham, West Midlands...","[{'label': 'display', 'lat': 52.48302813782334...",52.483028,-1.893479,NaN,B4 6QD,West Midlands,50c5f20ee4b049fc9923a8b9
2,The Jekyll & Hyde,Pub,28 Steelhouse Ln,GB,Birmingham,United Kingdom,at Printing House St,538,"[28 Steelhouse Ln (at Printing House St), Birm...","[{'label': 'display', 'lat': 52.48355982287995...",52.483560,-1.895151,NaN,B4 6BJ,West Midlands,4b1813bcf964a5209bcc23e3
3,Loki Wine Merchant & Tasting House,Wine Shop,36 Great Western Arcade,GB,Birmingham,United Kingdom,Off Colmore Row,762,"[36 Great Western Arcade (Off Colmore Row), Bi...","[{'label': 'display', 'lat': 52.48201493284286...",52.482015,-1.897398,City Core,B2 5HU,West Midlands,4f1dd5fb754a4cd5a2965094
4,The Wolf,Bar,2-10 Constitution Hill,GB,Birmingham,United Kingdom,NaN,869,"[2-10 Constitution Hill, Birmingham, West Midl...","[{'label': 'display', 'lat': 52.48649037415316...",52.486490,-1.901312,Jewellery Quarter,B19 3LY,West Midlands,58811e8e349355113b2413d6
5,200 Degrees Coffee,Coffee Shop,21-23 Colmore Row,GB,Birmingham,United Kingdom,NaN,828,"[21-23 Colmore Row, Birmingham, West Midlands,...","[{'label': 'display', 'lat': 52.48178393527601...",52.481784,-1.898328,NaN,B3 2BH,West Midlands,579f397538fae16662c83220
6,Original Patty Men,Burger Joint,Shaw's Passage,GB,Birmingham,United Kingdom,NaN,866,"[Shaw's Passage, Birmingham, West Midlands, B5...","[{'label': 'display', 'lat': 52.47851495905522...",52.478515,-1.890398,NaN,B5 5JG,West Midlands,566d8b96498e24e2ed979311
7,Asha's Contemporary Indian Cuisine,Indian Restaurant,12-22 Newhall St.,GB,Birmingham,United Kingdom,NaN,1059,"[12-22 Newhall St., Birmingham, West Midlands,...","[{'label': 'display', 'lat': 52.48139829496564...",52.481398,-1.901992,NaN,B3 3LX,West Midlands,4c0bdcd47e3fc9285ae5f582
8,Adams,Restaurant,"New Oxford House, 16 Waterloo St",GB,Birmingham,United Kingdom,NaN,1133,"[New Oxford House, 16 Waterloo St, Birmingham,...","[{'label': 'display', 'lat': 52.4797974141386,...",52.479797,-1.901482,NaN,B2 5UG,West Midlands,5165b0cee4b0c1c41ac988f8
9,Tilt,Gaming Cafe,2 City Arcade,GB,Birmingham,United Kingdom,Warwick Passage,898,"[2 City Arcade (Warwick Passage), Birmingham, ...","[{'label': 'display', 'lat': 52.47971465452974...",52.479715,-1.896372,City Core,B2 4TX,West Midlands,561b607d498e6653313c1ea0


##### Liverpool

In [37]:
addr3 = 'Liverpool, UK'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(addr3)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

53.407154 -2.991665


In [38]:
search_query = 'Bakery'
radius = 5000
LIMIT=50
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
results3 = requests.get(url).json()

In [39]:
venues = results3['response']['venues']
dataframe3 = json_normalize(venues)
dataframe3.head()
filtered_columns3 = ['name', 'categories'] + [col for col in dataframe3.columns if col.startswith('location.')] + ['id']
dataframe_filtered3 = dataframe3.loc[:, filtered_columns3]
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
dataframe_filtered3['categories'] = dataframe_filtered3.apply(get_category_type, axis=1)
dataframe_filtered3.columns = [column.split('.')[-1] for column in dataframe_filtered3.columns]

In [40]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Liverpool',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)
for lat, lng, label in zip(dataframe_filtered3.lat, dataframe_filtered3.lng, dataframe_filtered3.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
venues_map

In [41]:
#Explore Islington
latitude = 53.412064
longitude = -2.972211
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
results3a = requests.get(url).json()
items3a = results3a['response']['groups'][0]['items']
items3a[0]
df3a = json_normalize(items3a)
filt_col3a = ['venue.name', 'venue.categories'] + [col for col in df3a.columns if col.startswith('venue.location.')] + ['venue.id']
df_filt3a = df3a.loc[:, filt_col3a]
df_filt3a['venue.categories'] = df_filt3a.apply(get_category_type, axis=1)
df_filt3a.columns = [col.split('.')[-1] for col in df_filt3a.columns]
df_filt3a

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Walker Art Gallery,Art Gallery,William Brown St,GB,Liverpool,United Kingdom,at Cnr of Dale St & Islington,542,[William Brown St (at Cnr of Dale St & Islingt...,"[{'label': 'display', 'lat': 53.40984709471016...",53.409847,-2.979499,NaN,L3 8EL,NaN,4b681fb8f964a52028682be3
1,Lovelock's,Coffee Shop,"Unit 6, Old Haymarket",GB,Liverpool,United Kingdom,NaN,807,"[Unit 6, Old Haymarket, Liverpool, L1 6ER, Uni...","[{'label': 'display', 'lat': 53.408957, 'lng':...",53.408957,-2.983204,NaN,L1 6ER,Liverpool,576e6427498e95e8e6ec0bf6
2,Free State Kitchen,American Restaurant,1 Maryland St,GB,Liverpool,United Kingdom,off Hope St,1078,"[1 Maryland St (off Hope St), Liverpool, L1 9D...","[{'label': 'display', 'lat': 53.40243579821311...",53.402436,-2.970429,NaN,L1 9DE,NaN,50e9b53eebca6fb3830a4553
3,The Dead Crafty Beer Company,Beer Bar,92 Dale Street,GB,Liverpool,United Kingdom,Preston Street,918,"[92 Dale Street (Preston Street), Liverpool, L...","[{'label': 'display', 'lat': 53.40920568817392...",53.409206,-2.985202,Central Liverpool,L2 5TF,Liverpool,56c37a91cd1020ede50eae57
4,Elif,Turkish Restaurant,Bold Street,GB,Liverpool,United Kingdom,NaN,1054,"[Bold Street, Liverpool, L1, United Kingdom]","[{'label': 'display', 'lat': 53.40352293150271...",53.403523,-2.979076,NaN,L1,Liverpool,578d17ca498e0662a3168ffc
5,Leaf,Café,65-67 Bold St,GB,Liverpool,United Kingdom,NaN,1086,"[65-67 Bold St, Liverpool, L1 4EZ, United King...","[{'label': 'display', 'lat': 53.40286866580079...",53.402869,-2.977708,NaN,L1 4EZ,NaN,4ce58cac2b4e721ece5e352b
6,Five Guys,Burger Joint,"Former R&P, Queens Sq.",GB,Liverpool,United Kingdom,NaN,801,"[Former R&P, Queens Sq., Liverpool, L1 1RH, Un...","[{'label': 'display', 'lat': 53.40785255211248...",53.407853,-2.982007,NaN,L1 1RH,NaN,56659b3e498e5dafbd939e1c
7,St John's Gardens,Garden,Old Haymarket,GB,"Liverpool, Merseyside",United Kingdom,St John's Lane,708,"[Old Haymarket (St John's Lane), Liverpool, Me...","[{'label': 'display', 'lat': 53.40885393889672...",53.408854,-2.981431,NaN,NaN,NaN,4c13ab2ba9c220a14319569d
8,Ship and Mitre,Pub,133 Dale St,GB,Liverpool,United Kingdom,NaN,863,"[133 Dale St, Liverpool, L2 2JH, United Kingdom]","[{'label': 'display', 'lat': 53.40952119551682...",53.409521,-2.984511,NaN,L2 2JH,Liverpool,4b50bb7bf964a520d92f27e3
9,Kazimier Garden,Beer Garden,32 Seel St,GB,Liverpool,United Kingdom,NaN,1214,"[32 Seel St, Liverpool, L1 4BE, United Kingdom]","[{'label': 'display', 'lat': 53.40280450733662...",53.402805,-2.981879,NaN,L1 4BE,NaN,4ff75d1fe4b08a4c729f861e


##### Glasgow City

In [42]:
addr4 = 'Glasgow City, UK'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(addr4)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

55.85551155 -4.22623065725823


In [43]:
search_query = 'Bakery'
radius = 5000
LIMIT=50
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url
results4 = requests.get(url).json()

In [44]:
venues = results4['response']['venues']
dataframe4 = json_normalize(venues)
dataframe4.head()
filtered_columns4 = ['name', 'categories'] + [col for col in dataframe4.columns if col.startswith('location.')] + ['id']
dataframe_filtered4 = dataframe4.loc[:, filtered_columns4]
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
dataframe_filtered4['categories'] = dataframe_filtered4.apply(get_category_type, axis=1)
dataframe_filtered4.columns = [column.split('.')[-1] for column in dataframe_filtered4.columns]

In [45]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Glasgow',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)
for lat, lng, label in zip(dataframe_filtered4.lat, dataframe_filtered4.lng, dataframe_filtered4.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
venues_map

In [46]:
#Explore Garnethill
latitude = 55.866863
longitude = -4.263974
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
results4a = requests.get(url).json()
items4a = results4a['response']['groups'][0]['items']
items4a[0]
df4a = json_normalize(items4a)
filt_col4a = ['venue.name', 'venue.categories'] + [col for col in df4a.columns if col.startswith('venue.location.')] + ['venue.id']
df_filt4a = df4a.loc[:, filt_col4a]
df_filt4a['venue.categories'] = df_filt4a.apply(get_category_type, axis=1)
df_filt4a.columns = [col.split('.')[-1] for col in df_filt4a.columns]
df_filt4a

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Glasgow Film Theatre,Indie Movie Theater,12 Rose St,GB,Glasgow,United Kingdom,NaN,203,"[12 Rose St, Glasgow, Glasgow City, G3 6RB, Un...","[{'label': 'display', 'lat': 55.86572291734018...",55.865723,-4.261418,NaN,G3 6RB,Glasgow City,4b50c91df964a520753227e3
1,The Butterfly and the Pig,Bar,153 Bath St,GB,Glasgow,United Kingdom,NaN,329,"[153 Bath St, Glasgow, Glasgow City, G2 4SQ, U...","[{'label': 'display', 'lat': 55.86438793225795...",55.864388,-4.261081,NaN,G2 4SQ,Glasgow City,4b1fe8d7f964a520a32a24e3
2,citizenM Glasgow,Hotel,60 Renfrew St,GB,Glasgow,United Kingdom,Hope St,486,"[60 Renfrew St (Hope St), Glasgow, Glasgow Cit...","[{'label': 'display', 'lat': 55.86549315709696...",55.865493,-4.256572,NaN,G2 3BW,Glasgow City,4bded29717fdef3b21a84e36
3,The Pot Still,Whisky Bar,145 Hope St.,GB,Glasgow,United Kingdom,West Regent St.,583,"[145 Hope St. (West Regent St.), Glasgow, Glas...","[{'label': 'display', 'lat': 55.86291561558843...",55.862916,-4.257837,Merchant City,G2 2TH,Glasgow City,4bb7a8451261d13a09a7e798
4,Waterstones,Bookstore,153-157 Sauchiehall Street,GB,Glasgow,United Kingdom,NaN,411,"[153-157 Sauchiehall Street, Glasgow, Glasgow ...","[{'label': 'display', 'lat': 55.8648132949987,...",55.864813,-4.258491,NaN,G2 3EW,Glasgow City,4fbb9cf87beb19fa66216534
5,The State Bar,Bar,148 Holland St,GB,Glasgow,United Kingdom,NaN,198,"[148 Holland St, Glasgow, Glasgow City, G2 4NG...","[{'label': 'display', 'lat': 55.865662797896, ...",55.865663,-4.266318,NaN,G2 4NG,Glasgow City,4b607d0df964a52035e929e3
6,Mussel Inn,Seafood Restaurant,157 Hope St,GB,Glasgow,United Kingdom,NaN,626,"[157 Hope St, Glasgow, Glasgow City, G2 2UQ, U...","[{'label': 'display', 'lat': 55.862205, 'lng':...",55.862205,-4.258351,NaN,G2 2UQ,Glasgow City,4c151cf5f1e0b713f9f134bc
7,Blytheswood Hotel,Hotel,NaN,GB,NaN,United Kingdom,NaN,400,[United Kingdom],"[{'label': 'display', 'lat': 55.86339954225887...",55.863400,-4.262258,City Centre,NaN,NaN,4efdfa6bb6346feaeb0a3c54
8,Gin71,Cocktail Bar,71 Renfield St,GB,Glasgow,United Kingdom,NaN,638,"[71 Renfield St, Glasgow, Glasgow City, G2 1LP...","[{'label': 'display', 'lat': 55.86308210979272...",55.863082,-4.256290,NaN,G2 1LP,Glasgow City,536d3b5011d2fa1bd01363a6
9,King's Theatre,Theater,297 Bath St.,GB,Glasgow,United Kingdom,Elmbank St.,338,"[297 Bath St. (Elmbank St.), Glasgow, Glasgow ...","[{'label': 'display', 'lat': 55.8649981017317,...",55.864998,-4.268246,NaN,G2 4JN,Glasgow City,4ba2864af964a520f00038e3


In [49]:
#Explore Townhead
latitude = 55.866139
longitude = -4.242650
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
results4b = requests.get(url).json()
items4b = results4b['response']['groups'][0]['items']
items4b[0]
df4b = json_normalize(items4b)
filt_col4b = ['venue.name', 'venue.categories'] + [col for col in df4b.columns if col.startswith('venue.location.')] + ['venue.id']
df_filt4b = df4b.loc[:, filt_col4b]
df_filt4b['venue.categories'] = df_filt4b.apply(get_category_type, axis=1)
df_filt4b.columns = [col.split('.')[-1] for col in df_filt4b.columns]
df_filt4b

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Tiso Glasgow Outdoor Experience,Sporting Goods Shop,50 Couper St,GB,Glasgow,United Kingdom,Kyle St,261,"[50 Couper St (Kyle St), Glasgow, Glasgow City...","[{'label': 'display', 'lat': 55.8677918710161,...",55.867792,-4.245630,NaN,G4 0DL,Glasgow City,4bb215dff964a5209cb73ce3
1,Paesano Pizza,Pizza Place,94 Miller St,GB,Glasgow,United Kingdom,Ingram St,875,"[94 Miller St (Ingram St), Glasgow, Glasgow Ci...","[{'label': 'display', 'lat': 55.85972127962135...",55.859721,-4.250754,NaN,G1 1DT,Glasgow City,55ca18f2498e89eef195151d
2,DogHouse Merchant City,Beer Bar,99 Hutcheson St,GB,Glasgow,United Kingdom,NaN,819,"[99 Hutcheson St, Glasgow, Glasgow City, G1 1S...","[{'label': 'display', 'lat': 55.85937678742589...",55.859377,-4.247850,NaN,G1 1SN,Glasgow City,53e52950498ee2c0974731b8
3,citizenM Glasgow,Hotel,60 Renfrew St,GB,Glasgow,United Kingdom,Hope St,872,"[60 Renfrew St (Hope St), Glasgow, Glasgow Cit...","[{'label': 'display', 'lat': 55.86549315709696...",55.865493,-4.256572,NaN,G2 3BW,Glasgow City,4bded29717fdef3b21a84e36
4,Hutchesons Glasgow,Steakhouse,158 Ingram St,GB,Glasgow,United Kingdom,John St,777,"[158 Ingram St (John St), Glasgow, Glasgow Cit...","[{'label': 'display', 'lat': 55.85979954006429...",55.859800,-4.247892,NaN,G1 1EJ,Glasgow City,531864cc498e1c272ef9f65a
5,The LEGO Store,Toy / Game Store,220 Buchanan St,GB,Glasgow,United Kingdom,NaN,643,"[220 Buchanan St, Glasgow, Glasgow City, G1 2F...","[{'label': 'display', 'lat': 55.86369085872041...",55.863691,-4.251984,NaN,G1 2FF,Glasgow City,518cdbac498edf83d44e54fb
6,The Pot Still,Whisky Bar,145 Hope St.,GB,Glasgow,United Kingdom,West Regent St.,1014,"[145 Hope St. (West Regent St.), Glasgow, Glas...","[{'label': 'display', 'lat': 55.86291561558843...",55.862916,-4.257837,Merchant City,G2 2TH,Glasgow City,4bb7a8451261d13a09a7e798
7,Gin71,Cocktail Bar,71 Renfield St,GB,Glasgow,United Kingdom,NaN,917,"[71 Renfield St, Glasgow, Glasgow City, G2 1LP...","[{'label': 'display', 'lat': 55.86308210979272...",55.863082,-4.256290,NaN,G2 1LP,Glasgow City,536d3b5011d2fa1bd01363a6
8,Wilson Street Pantry,Breakfast Spot,Wilson Street,GB,Glasgow,United Kingdom,NaN,902,"[Wilson Street, Glasgow, Glasgow City, United ...","[{'label': 'display', 'lat': 55.85825085698097...",55.858251,-4.245971,Merchant City,NaN,Glasgow City,54c7cade498e300defb0985f
9,Café Gandolfi,Scottish Restaurant,64 Albion St,GB,Glasgow,United Kingdom,NaN,882,"[64 Albion St, Glasgow, G1 1NY, United Kingdom]","[{'label': 'display', 'lat': 55.85827329529071...",55.858273,-4.244358,Merchant City,G1 1NY,NaN,4b50d353f964a520df3327e3
